In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
filepath = "data/HW1_var_5.csv"
data_df = pd.read_csv(filepath, sep=';')
print("data_df shape:", data_df.shape)
data_df.head()

data_df shape: (10243, 44)


,Номер варианта,ID,INCOME_BASE_TYPE,CREDIT_PURPOSE,INSURANCE_FLAG,DTI,SEX,FULL_AGE_CHILD_NUMBER,DEPENDANT_NUMBER,EDUCATION,...,own_closed,min_MnthAfterLoan,max_MnthAfterLoan,dlq_exist,thirty_in_a_year,sixty_in_a_year,ninety_in_a_year,thirty_vintage,sixty_vintage,ninety_vintage
0,5,1000005,Поступление зарплаты на счет,Покупка недвижимости/ строительство,1,0.26,мужской,0,0,среднее,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5,1000025,Поступление зарплаты на счет,Покупка автомобиля,1,0.45,мужской,1,0,высшее,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5,1000045,2НДФЛ,Ремонт,1,0.49,мужской,0,0,высшее,...,0.0,66.0,66.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,5,1000065,Форма банка (без печати работодателя),Ремонт,1,0.38,мужской,1,0,Высшее/Второе высшее/Ученая степень,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,1000085,Поступление зарплаты на счет,Ремонт,0,0.16,мужской,1,0,высшее,...,0.0,49.0,86.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
